In [75]:
from mpi4py import MPI
from basix.ufl import mixed_element
#import basix
import numpy as np
from dolfinx import mesh, io
from dolfinx.fem import Function, FunctionSpace, dirichletbc, locate_dofs_topological, form
import ufl
from ufl import grad, div, nabla_grad, dx, inner, dot, Measure, SpatialCoordinate, TestFunctions, TrialFunctions, div, exp, inner

domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)

from basix.ufl import element, mixed_element
P2 = element("Lagrange", domain.basix_cell(), 2, rank=1)
P1 = element("Lagrange", domain.basix_cell(), 1)
TH = mixed_element([P2, P1])
V = FunctionSpace(domain, TH)
(v_a, p_a) = TrialFunctions(V)
(v_t, p_t) = TestFunctions(V)

x = SpatialCoordinate(domain)




ModuleNotFoundError: No module named 'basix.ufl'

In [ ]:
def Left(x):
        return np.isclose(x[0], 0) and (0 <= x[1] <= 1)
def bottom(x):
        return np.isclose(x[1], 0) and (0 <= x[0] <= 1)
def Top(x):
        return np.isclose(x[0], 1) and (0 <= x[1] <= 1)
def Right(x):
        return np.isclose(x[1], 1) and (0 <= x[0] <= 1)

dofs_TopRightBoundary = locate_dofs_geometrical(V, Top, Right)
dofs_SideBoundary = locate_dofs_geometrical(V, Left)
dofs_BottomBoundary = locate_dofs_geometrical(V, Bottom)

def u_exact(x):
    theta = atan_2(x[1], x[0])
    radius = (x[0]**2 + x[1]**2)**0.5
    Ue = -U
    theta0 = 0.5*pi
    f = Ue*(-theta0**2*sin(theta) + theta0*theta*sin(theta) \
           - sin(theta0)*cos(theta0)*theta*sin(theta) \
           + sin(theta0)**2*theta*cos(theta)) \
           /(theta0**2 - sin(theta0)**2)
    dfdt = Ue*(-theta0**2*cos(theta) + theta0*sin(theta) \
              + theta0*theta*cos(theta) - sin(theta0)*cos(theta0)*sin(theta) \
              - sin(theta0)*cos(theta0)*theta*cos(theta) \
              + sin(theta0)**2*cos(theta) - sin(theta0)**2*theta*sin(theta)) \
           /(theta0**2 - sin(theta0)**2)
    psie = radius*f
    vre = dfdt
    vte = -f
    ve = as_vector([vre*cos(theta) - vte*sin(theta),
                    vre*sin(theta) + vte*cos(theta)])
    return 
u_bc = Function(V)
u_bc.interpolate(u_exact)
bc_TopRight = dirichletbc(u_bc, dofs_TopRightBoundary)
bc_Side = dirichletbc(u)

n = FacetNormal(mesh)

K = inner(sym(grad(v_t)), sym(grad(v_a)))*dx 
G = -div(v_t)*p_a*dx
D = -p_t*div(v_a)*dx
S=K+G+D
L = Constant(domain, ScalarType(0.0)) * v_t * dx

#f = T_t*rhsf*dx + T_t*inner(gN, n)*ds
problem = LinearProblem(S, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
u_i = problem.solve()

